# PIFuHD: High-Resolution 3D Human Digitization from A Single Image.

**Original Repo**: https://github.com/facebookresearch/pifuhd

**Provided By**: https://github.com/BreezeWhite/pifuhd

**Paper**: https://arxiv.org/pdf/2004.00452.pdf

In [ ]:
#@title Setup

!git clone https://github.com/BreezeWhite/pifuhd

%cd pifuhd
!sh ./scripts/download_trained_model.sh


import sys
import torch


pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

!pip install git+https://github.com/BreezeWhite/segpeo

In [ ]:
#@title Upload Image

import os
import IPython.display as dsp
from pathlib import Path
from google.colab import files


ff = files.upload()
img_path = Path(list(ff.keys())[0])
img_basename = img_path.name
img_name = img_path.stem

img_store_folder = Path('images') / img_name
img_store_path = img_store_folder / img_basename
img_store_path.parent.mkdir(parents=True, exist_ok=True)
os.rename(img_path, img_store_path)

dsp.display(dsp.Image(img_store_path))

In [ ]:
#@title Run

!segpeo --image-path {img_store_path}

!python -m apps.simple_test \
  -r 256 \
  --use_rect \
  -i {img_store_folder}


from lib.colab_util import generate_video_from_obj, set_renderer, video


# output pathes
obj_path = f'./results/pifuhd_final/recon/result_{img_name}_256.obj'
out_img_path = f'./results/pifuhd_final/recon/result_{img_name}_256.png'
video_path = f'./results/pifuhd_final/recon/result_{img_name}_256.mp4'
video_display_path = f'./results/pifuhd_final/result_{img_name}_256_display.mp4'

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# we cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)